<a href="https://colab.research.google.com/github/koki-223/my-first-Repository/blob/main/Copy_of_Lab04_Naive_Bayes_Grid_and_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [2]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [10]:
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

print(classification_report(ytest, ypred, target_names=test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.83      0.24      0.37        21
           comp.graphics       0.83      0.24      0.37        21
 comp.os.ms-windows.misc       0.71      0.65      0.68        26
comp.sys.ibm.pc.hardware       0.73      0.56      0.63        34
   comp.sys.mac.hardware       0.73      0.79      0.76        34
          comp.windows.x       0.95      0.81      0.88        26
            misc.forsale       1.00      0.50      0.67        22
               rec.autos       0.67      1.00      0.80        28
         rec.motorcycles       0.96      0.73      0.83        33
      rec.sport.baseball       0.91      0.84      0.88        25
        rec.sport.hockey       0.85      0.85      0.85        27
               sci.crypt       0.58      0.95      0.72        20
         sci.electronics       0.56      0.38      0.45        24
                 sci.med       0.81      0.57      0.67        23
         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [11]:
# สร้าง Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# กำหนดพารามิเตอร์สำหรับ RandomizedSearchCV
param_dist = {
    'tfidf__max_df': uniform(0.5, 1.0),         # ทดลองค่าระหว่าง 0.5 ถึง 1.0
    'tfidf__min_df': [1, 2, 3, 5],
    'tfidf__ngram_range': [(1, 1), (1, 2)],     # unigram หรือ bigram
    'nb__alpha': uniform(0.0, 1.0)              # smoothing parameter
}

# Randomized Search
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,         # จำนวนการทดลอง
    cv=3,              # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# ฝึกโมเดล
search.fit(Xtrain, ytrain)

# ทำนายและประเมินผล
ypred = search.predict(Xtest)
print("Best Parameters:", search.best_params_)
print("\nClassification Report:")
print(classification_report(ytest, ypred, target_names=test.target_names))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, ra

Best Parameters: {'nb__alpha': np.float64(0.18340450985343382), 'tfidf__max_df': np.float64(0.8042422429595377), 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}

Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.73      0.52      0.61        21
           comp.graphics       0.56      0.48      0.51        21
 comp.os.ms-windows.misc       0.54      0.58      0.56        26
comp.sys.ibm.pc.hardware       0.73      0.71      0.72        34
   comp.sys.mac.hardware       0.83      0.88      0.86        34
          comp.windows.x       0.89      0.62      0.73        26
            misc.forsale       1.00      0.82      0.90        22
               rec.autos       0.74      1.00      0.85        28
         rec.motorcycles       0.90      0.82      0.86        33
      rec.sport.baseball       0.91      0.84      0.88        25
        rec.sport.hockey       0.84      1.00      0.92        27
               sci.c

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [13]:
from sklearn.metrics import f1_score

# Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Grid Search: ทดลองหลายค่า alpha
param_grid = {
    'nb__alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
}

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(Xtrain, ytrain)

# ทำนายและคำนวณ f1_macro
ypred_grid = grid.predict(Xtest)
f1_grid = f1_score(ytest, ypred_grid, average='macro')

print("🔍 Grid Search Best alpha:", grid.best_params_['nb__alpha'])
print("🎯 f1_macro on test set (Grid Search):", round(f1_grid, 4))

🔍 Grid Search Best alpha: 0.001
🎯 f1_macro on test set (Grid Search): 0.7442


In [14]:
# Random Search: ทดลอง alpha ในช่วงกว้างขึ้น
param_dist = {
    'nb__alpha': uniform(0.001, 10.0)
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    random_state=42
)

random_search.fit(Xtrain, ytrain)

# ทำนายและคำนวณ f1_macro
ypred_random = random_search.predict(Xtest)
f1_random = f1_score(ytest, ypred_random, average='macro')

print("🔍 Random Search Best alpha:", round(random_search.best_params_['nb__alpha'], 4))
print("🎯 f1_macro on test set (Random Search):", round(f1_random, 4))

🔍 Random Search Best alpha: 0.2068
🎯 f1_macro on test set (Random Search): 0.6862
